In [29]:
# Notebook til at forsøge at fremskrive vaccinationsdata
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps

cmap = cm.get_cmap('Dark2',7)

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))


import os
import math

from datetime import date
import datetime

saveFigures = True
print('saveFigures is set to: '+str(saveFigures))
print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[meanWidth-1:]


saveFigures is set to: True
Done loading packages


In [30]:
ssidatapath = "ssi_vacc"
rootdir = os.getcwd() +"\\" + ssidatapath

for f in os.scandir(rootdir):
    curPath = f.path
    latestdir = curPath
    latestDate = pd.to_datetime(curPath[-10:])

print(latestdir)
print(latestDate)

filename = 'Vaccinerede_pr_uge_pr_maalgruppe.csv'
curPath = latestdir+"/Vaccine_maalgrupper_DB/" + filename

df_weekly = pd.read_csv(curPath,delimiter = ';',dtype=str,encoding='latin-1')


filename = 'PaabegVacc_daek_DK_prdag.csv'
# curPath = "D:/Pandemix/Github/DanskeData/ssi_vacc/SSI_vacc_2021-05-25/Vaccine_DB/" + filename
curPath = latestdir+"/Vaccine_DB/" + filename


# df = pd.read_csv(curPath,delimiter = ',',dtype=str,encoding='latin-1')
df = pd.read_csv(curPath,delimiter = ';',dtype=str,encoding='latin-1')


filename = 'FaerdigVacc_daekning_DK_prdag.csv'
curPath = latestdir+"/Vaccine_DB/" + filename

df_full = pd.read_csv(curPath,delimiter = ';',dtype=str,encoding='latin-1')


c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData\ssi_vacc\SSI_vacc_2021-08-03
2021-08-03 00:00:00


In [31]:
datesOnce = pd.to_datetime(df.Vaccinedato)
datesFull = pd.to_datetime(df_full.Vaccinedato)
vaccOnce = pd.to_numeric(df['Kumuleret antal påbegyndt vacc.'])
vaccFull = pd.to_numeric(df_full['Kumuleret antal færdigvacc.'])


fig,ax1 = plt.subplots()

ax1.plot(datesOnce,vaccOnce)
ax1.plot(datesFull,vaccFull)

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:

# Drop the three extra empty rows are the top
df_weekly.drop([0,1,2],axis=0,inplace=True)

allDates = df_weekly.Ugenummer

curWeekDays = df_weekly.Ugenummer.apply(lambda x: np.datetime64(datetime.datetime.strptime(x + '-1', "%Y-W%W-%w")))

allVaccOnce = pd.to_numeric(df_weekly['Dækning førstegangsvaccinerede (kumulativt i %)'])
allVaccFull = pd.to_numeric(df_weekly['Dækning færdigvaccinerede (kumulativt i %)'])
allGroups = df_weekly['Målgruppe'].unique()

# Move groups around to have correct order
allGroups.sort()
allGroups = np.roll(allGroups,-8)
allGroups = np.insert(allGroups,0,allGroups[-8])
allGroups = np.delete(allGroups,-8)
allGroups = np.append(allGroups,allGroups[-8])
allGroups = np.delete(allGroups,-9)
allGroups


array(['1. Plejehjemsbeboere',
       '2. Borgere > 65, praktisk hjælp og personlig pleje',
       '3. Borgere fra årgang 1936 og derunder (85 år og ældre)',
       '4. Personale i sundhedsvæsenet og dele af socialvæsenet',
       '5. Udvalgte patienter med særligt øget risiko',
       '6. Udvalgte pårørende til personer med særligt øget risiko',
       '7. Personer fra årgang 1937-1941',
       '8. Personer fra årgang 1942-1946',
       '9. Personer fra årgang 1947-1956',
       '10A. Personer fra årgang 1957-1961',
       '10B. Personer fra årgang 1962-1966',
       '10C. Personer fra årgang 1967-1971',
       '10D1. Personer fra årgang 1972-1976 og årgang 2002-2005',
       '10D2. Personer fra årgang 1977-1981 og årgang 1997-2001',
       '10D3. Personer fra årgang 1982-1986 og årgang 1992-1996',
       '10D4. Personer fra årgang 1987-1991', 'Resterende'], dtype=object)

In [33]:
# Hard-coded fix of the missing data-points
group5Len = len(df_weekly[df_weekly['Målgruppe'] == allGroups[5]])
otherGroupLen = len(df_weekly[df_weekly['Målgruppe'] == allGroups[2]])
if (group5Len == otherGroupLen - 3):

    rowToCopy = df_weekly.loc[57]
    toAdd1 = rowToCopy.copy()
    toAdd2 = rowToCopy.copy()
    toAdd3 = rowToCopy.copy()
    toAdd1.Ugenummer = '2021-W03'
    toAdd2.Ugenummer = '2021-W04'
    toAdd3.Ugenummer = '2021-W06'

    df_weekly = df_weekly.append(toAdd1)
    df_weekly = df_weekly.append(toAdd2)
    df_weekly = df_weekly.append(toAdd3)

# Sort so order is correct
df_weekly.sort_values(by=['Ugenummer','Målgruppe'],inplace=True)

In [34]:
def logiGrowth(x,K,r,x0):
    
    curBot = 1 + ((K-x0)/x0) * np.exp(-r*x)
    
    return K/curBot

In [35]:
xRange = np.arange(0,20,0.1)

# testPars = (0.9,0.5,0.01)

testY = logiGrowth(xRange,0.9,0.5,0.01)

fig,ax1 = plt.subplots()

ax1.plot(xRange,testY)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:


# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,ax1 = plt.subplots()


for k in range(len(allGroups)-5,len(allGroups)):
# for k in range(0,len(allGroups)):
    curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[k]]

    curLabel = allGroups[k]
    # curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[8]]
    # curLabel = allGroups[8]

    ageWeekDays = curdf.Ugenummer.apply(lambda x: np.datetime64(datetime.datetime.strptime(x + '-1', "%Y-W%W-%w"))).values

    ageVaccOnce = pd.to_numeric(curdf['Dækning førstegangsvaccinerede (kumulativt i %)']).values

    datToUse = 5

    xPlot = ageWeekDays[datToUse:]
    yPlot = ageVaccOnce[datToUse:]
    xVals = ageWeekDays[datToUse:]
    xVals = np.array(xVals,dtype='datetime64[D]')
    xVals = xVals-xVals[0]
    xVals = xVals/np.timedelta64(1,'D')

    from scipy.optimize import curve_fit
    popt, pcov = curve_fit(logiGrowth,xVals,yPlot)


    ax1.plot(xPlot,logiGrowth(xVals,*popt),'k')
    ax1.plot(xPlot,yPlot,'*',label=curLabel+f' Max: {popt[0]:2.0f}')

ax1.set_ylim([0,100])
ax1.legend(fontsize=10)

ax1.set_xlim(left=np.datetime64('2021-03-01'))
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-34-acd433c12699>:3: RuntimeWarning: overflow encountered in exp
  curBot = 1 + ((K-x0)/x0) * np.exp(-r*x)


In [78]:
# Try an alternative function
def newFunc(x,K,r,p0):
    return  p0* np.exp(-r*x) + K
    # return K*(1-np.exp(-r*x))

fig,ax1 = plt.subplots()

xRange = np.arange(0,37)

ax1.plot(xRange,newFunc(xRange,30,0.1,-20))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [65]:


# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,ax1 = plt.subplots(figsize=(18,10))

cmap = cm.get_cmap('twilight',len(allGroups))

# for k in range(len(allGroups)-8,len(allGroups)-3):
for k in range(len(allGroups)-12,len(allGroups)-1):
# for k in range(0,len(allGroups)):

    datToUse = -12
    
    
    curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[k]]

    curLabel = allGroups[k]
    # curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[8]]
    # curLabel = allGroups[8]

    ageWeekDays = curdf.Ugenummer.apply(lambda x: np.datetime64(datetime.datetime.strptime(x + '-1', "%Y-W%W-%w"))).values

    ageVaccOnce = pd.to_numeric(curdf['Dækning førstegangsvaccinerede (kumulativt i %)']).values
    
    firstValsToUse = 40
    firstIndexToUse = np.where(ageVaccOnce > firstValsToUse)[0][0]
    datToUse = firstIndexToUse
    # datToUse = -8
    xPlot = ageWeekDays[datToUse:]
    yPlot = ageVaccOnce[datToUse:]
    
    xVals = xPlot
    xVals = np.array(xVals,dtype='datetime64[D]')
    xVals = xVals-xVals[0]
    xVals = xVals/np.timedelta64(1,'D')
    
    yFit = 100-yPlot

    from scipy.optimize import curve_fit
    # popt, pcov = curve_fit(newFunc,xVals,yFit)
    p0 = [20,0.06,70]
    popt, pcov = curve_fit(newFunc,xVals,yFit,p0=p0)


    ax1.plot(xPlot,100-newFunc(xVals,*popt),color=cmap(k))
    ax1.plot(xPlot,100-yFit,'*',label=curLabel+f'\nMaksimal dækning (fremskrivning): {100-popt[0]:2.0f}%',color=cmap(k))
    ax1.plot(ageWeekDays[:datToUse+1],ageVaccOnce[:datToUse+1],':.',color=cmap(k))
    
    # xPred = np.arange(xPlot[0] ,xPlot[0] + np.timedelta64(120,'D'),np.timedelta64(7,'D'))
    lastPredDay = np.datetime64('2021-10-10')
    xPred = np.arange(xPlot[0] ,lastPredDay,np.timedelta64(7,'D'))
    xPredMod = (xPred-xPred[0]) / np.timedelta64(1,'D')
    
    ax1.plot(xPred,100-newFunc(xPredMod,*popt),'--',color=cmap(k))
    

ax1.plot(np.nan,np.nan,'k.:',label='Data')    
ax1.plot(np.nan,np.nan,'k*',label='Data (benyttet i model)')    
ax1.plot(np.nan,np.nan,'k',label='Model')
ax1.plot(np.nan,np.nan,'k--',label='Model fremskrivning')    

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

ax1.set_ylim([0,100])
ax1.legend(fontsize=12, bbox_to_anchor=(1.05, 1), loc='upper left')

ax1.set_ylabel('Andel af målgruppe påbegyndt vaccination [%]')

ax1.set_xlim(left=np.datetime64('2021-03-01'))
ax1.set_xlim(right=np.datetime64('2021-09-14'))
plt.tight_layout()

if saveFigures:
    plt.savefig('figsVacc/VaccinationFremskrivning')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:


# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
# fig,ax1 = plt.subplots()
fig,ax1 = plt.subplots(figsize=(18,10))


cmap = cm.get_cmap('twilight',len(allGroups))

# for k in range(len(allGroups)-8,len(allGroups)-3):
for k in range(len(allGroups)-12,len(allGroups)-4):
# for k in range(0,len(allGroups)):

    datToUse = -12
    
    
    curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[k]]

    curLabel = allGroups[k]
    # curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[8]]
    # curLabel = allGroups[8]

    ageWeekDays = curdf.Ugenummer.apply(lambda x: np.datetime64(datetime.datetime.strptime(x + '-1', "%Y-W%W-%w"))).values

    ageVaccOnce = pd.to_numeric(curdf['Dækning færdigvaccinerede (kumulativt i %)']).values
    
    firstValsToUse = 50
    firstIndexToUse = np.where(ageVaccOnce > firstValsToUse)[0][0]
    datToUse = firstIndexToUse
    # datToUse = -8
    xPlot = ageWeekDays[datToUse:]
    yPlot = ageVaccOnce[datToUse:]
    
    xVals = xPlot
    xVals = np.array(xVals,dtype='datetime64[D]')
    xVals = xVals-xVals[0]
    xVals = xVals/np.timedelta64(1,'D')
    
    yFit = 100-yPlot

    from scipy.optimize import curve_fit
    # popt, pcov = curve_fit(newFunc,xVals,yFit)
    p0 = [20,0.06,70]
    popt, pcov = curve_fit(newFunc,xVals,yFit,p0=p0)


    ax1.plot(xPlot,100-newFunc(xVals,*popt),color=cmap(k))
    ax1.plot(xPlot,100-yFit,'*',label=curLabel+f'\nMaksimal dækning (fremskrivning): {100-popt[0]:2.0f}%',color=cmap(k))
    ax1.plot(ageWeekDays[:datToUse+1],ageVaccOnce[:datToUse+1],':.',color=cmap(k))
    
    # xPred = np.arange(xPlot[0] ,xPlot[0] + np.timedelta64(120,'D'),np.timedelta64(7,'D'))
    lastPredDay = np.datetime64('2021-10-10')
    xPred = np.arange(xPlot[0] ,lastPredDay,np.timedelta64(7,'D'))
    xPredMod = (xPred-xPred[0]) / np.timedelta64(1,'D')
    
    ax1.plot(xPred,100-newFunc(xPredMod,*popt),'--',color=cmap(k))
    
    
ax1.plot(np.nan,np.nan,'k.:',label='Data')    
ax1.plot(np.nan,np.nan,'k*',label='Data (benyttet i model)')    
ax1.plot(np.nan,np.nan,'k',label='Model')
ax1.plot(np.nan,np.nan,'k--',label='Model fremskrivning')    


ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

ax1.set_ylim([0,100])
ax1.legend(fontsize=12, bbox_to_anchor=(1.05, 1), loc='upper left')

ax1.set_ylabel('Andel af målgruppe færdiggjort vaccination [%]')

ax1.set_xlim(left=np.datetime64('2021-03-01'))
ax1.set_xlim(right=np.datetime64('2021-09-14'))
plt.tight_layout()

if saveFigures:
    plt.savefig('figsVacc/VaccinationFremskrivning_FuldtVaccineret')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\rakrpe\Anaconda3\envs\main\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [40]:
# Load nøgletal
filename = 'Noegletal_vacc_daekning.csv'
curPath = latestdir+"/Vaccine_maalgrupper_DB/" + filename

df_noegle = pd.read_csv(curPath,delimiter = ';',dtype=str,encoding='latin-1')

In [68]:
# print(allGroups)
# print(len(allGroups))
hardcodedMaxVacc = 97*np.ones((17,))

hardcodedMaxVacc[ 3] = 99

# hardcodedMaxVacc[ 9] = 92
# hardcodedMaxVacc[10] = 91
# hardcodedMaxVacc[11] = 89
# hardcodedMaxVacc[12] = 85
# hardcodedMaxVacc[13] = 81
# hardcodedMaxVacc[14] = 74
# hardcodedMaxVacc[15] = 74
# hardcodedMaxVacc[16] = 10

# hardcodedMaxVacc[ 9] = 92 + 2
# hardcodedMaxVacc[10] = 91 + 2
# hardcodedMaxVacc[11] = 89 + 2
# hardcodedMaxVacc[12] = 85 + 2
# hardcodedMaxVacc[13] = 81 + 2
# hardcodedMaxVacc[14] = 74 + 2
# hardcodedMaxVacc[15] = 74 + 2
# hardcodedMaxVacc[16] = 10 + 10


hardcodedMaxVacc[ 9] = 93 + 2
hardcodedMaxVacc[10] = 91 + 2
hardcodedMaxVacc[11] = 89 + 2
hardcodedMaxVacc[12] = 85 + 2
hardcodedMaxVacc[13] = 80 + 2
hardcodedMaxVacc[14] = 70 + 2
hardcodedMaxVacc[15] = 66 + 2
hardcodedMaxVacc[16] = 20

# hardcodedMaxVacc[ 9] = 92
# hardcodedMaxVacc[10] = 91
# hardcodedMaxVacc[11] = 89
# hardcodedMaxVacc[12] = 85
# hardcodedMaxVacc[13] = 85
# hardcodedMaxVacc[14] = 85
# hardcodedMaxVacc[15] = 85
# hardcodedMaxVacc[16] = 20


np.stack([allGroups, hardcodedMaxVacc],axis=1)

array([['1. Plejehjemsbeboere', 97.0],
       ['2. Borgere > 65, praktisk hjælp og personlig pleje', 97.0],
       ['3. Borgere fra årgang 1936 og derunder (85 år og ældre)', 97.0],
       ['4. Personale i sundhedsvæsenet og dele af socialvæsenet', 99.0],
       ['5. Udvalgte patienter med særligt øget risiko', 97.0],
       ['6. Udvalgte pårørende til personer med særligt øget risiko',
        97.0],
       ['7. Personer fra årgang 1937-1941', 97.0],
       ['8. Personer fra årgang 1942-1946', 97.0],
       ['9. Personer fra årgang 1947-1956', 97.0],
       ['10A. Personer fra årgang 1957-1961', 95.0],
       ['10B. Personer fra årgang 1962-1966', 93.0],
       ['10C. Personer fra årgang 1967-1971', 91.0],
       ['10D1. Personer fra årgang 1972-1976 og årgang 2002-2005', 87.0],
       ['10D2. Personer fra årgang 1977-1981 og årgang 1997-2001', 82.0],
       ['10D3. Personer fra årgang 1982-1986 og årgang 1992-1996', 72.0],
       ['10D4. Personer fra årgang 1987-1991', 68.0],
       

In [69]:
numNotVaccedPred = []
numVaccedPred =[]

realOnce = []
realFull = []
allPop = []
for k in np.arange(len(allGroups)):
    curGroup = allGroups[k]
    thisdf = df_noegle[df_noegle['Målgruppe'] == curGroup]
    curPop = thisdf.Population.astype(int).sum()
    
    allPop.append(curPop)
    
    curOnce = thisdf['Antal førstevaccinerede'].astype(int).sum()
    curFull = thisdf['Antal færdigvaccinerede'].astype(int).sum()
    
    
    realOnce.append(curOnce)
    realFull.append(curFull)
    
    curNotVaccedPred = curPop * ((100 -  hardcodedMaxVacc[k])/100)
    curVaccedPred = curPop * hardcodedMaxVacc[k]/100
    numNotVaccedPred.append(curNotVaccedPred)
    numVaccedPred.append(curVaccedPred)
# curPop
# numNotVaccedPred

fig,ax1 = plt.subplots(figsize=(15,18))

totUnVaccPred = np.sum(numNotVaccedPred)
totVaccPred = np.sum(numVaccedPred)
totOnceReal = np.sum(realOnce)
totFullReal = np.sum(realFull)

# ax1.bar(allGroups,numNotVaccedPred,color='k',label=f'Ikke vaccineret: {totUnVaccPred:8.0f}')
# ax1.bar(allGroups,numVaccedPred,bottom=numNotVaccedPred,color='green',label=f'Vaccineret: {totVaccPred:15.0f}')

ax1.bar(allGroups,allPop,color='k',label=f'Uvaccineret (fremskrivning): {totUnVaccPred:8.0f}')
ax1.bar(allGroups,numVaccedPred,color='m',label=f'Vaccineret (fremskrivning): {totVaccPred:8.0f}')
ax1.bar(allGroups,realOnce,color='xkcd:light green',label=f'Første dosis: {totOnceReal:8.0f}')
ax1.bar(allGroups,realFull,color='g',label=f'Fuldt vaccineret: {totFullReal:8.0f}')

for k in range(len(allGroups)):
    # ax1.text(allGroups[k],allPop[k],allPop[k],ha='center',va='bottom')
    
    curRealOnce = realOnce[k]/allPop[k]
    curRealFull = realFull[k]/allPop[k]
    curPred = numVaccedPred[k]/allPop[k]
    curNon = (allPop[k]-numVaccedPred[k])/allPop[k]
    
    
    props = dict(boxstyle='round', facecolor='grey', alpha=0.75)
    
    dy = 0.18*1e5    
    ax1.text(allGroups[k],allPop[k]+0*dy,f'{100*curRealFull:2.0f}%',color='xkcd:dark green',ha='center',va='bottom')
    ax1.text(allGroups[k],allPop[k]+1*dy,f'{100*curRealOnce:2.0f}%',color='xkcd:green',ha='center',va='bottom')
    ax1.text(allGroups[k],allPop[k]+2*dy,f'{100*curPred:2.0f}%',color='m',ha='center',va='bottom')
    # ax1.text(allGroups[k],allPop[k]+3*dy,f'{100*curNon:2.0f}%',color='k',ha='center',va='bottom')

ax1.legend()

ax1.set_xticklabels(allGroups, rotation = 90, fontsize=12)
plt.tight_layout()

# print(totVaccPred/(totUnVaccPred+totVaccPred))
# print(totVaccPred/np.sum(allPop))
# print(np.sum(realOnce)/np.sum(allPop))

percVaccReal = np.sum(realOnce)/np.sum(allPop)
percVaccPred = totVaccPred/np.sum(allPop)

textToShow = f'Faktiske vaccinerede: {100*percVaccReal:2.1f}%\nFremskrivning: {100*percVaccPred:2.1f}%'
ax1.text(allGroups[7],7.5*1e5,textToShow,fontsize=24,ha='right')

ax1.set_ylim([0,1.05e6])

if saveFigures:
    plt.savefig('figsVacc/VaccinationFremskrivningBar')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-69-65a0fb99c057>:62: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(allGroups, rotation = 90, fontsize=12)


In [70]:
dfAges = pd.read_excel('DKfolketal2021clean.xlsx')
DK_age = []
DK_age.append(dfAges.iloc[0:10].Antal.sum())
DK_age.append(dfAges.iloc[10:20].Antal.sum())
DK_age.append(dfAges.iloc[20:30].Antal.sum())
DK_age.append(dfAges.iloc[30:40].Antal.sum())
DK_age.append(dfAges.iloc[40:50].Antal.sum())
DK_age.append(dfAges.iloc[50:60].Antal.sum())
DK_age.append(dfAges.iloc[60:70].Antal.sum())
DK_age.append(dfAges.iloc[70:80].Antal.sum())
DK_age.append(dfAges.iloc[80:90].Antal.sum())
DK_age.append(dfAges.iloc[90:].Antal.sum())
DK_age = np.array(DK_age)



In [76]:
dfAges['VaccProc'] = 0.95
# dfAges['VaccPred'] = dfAges.Antal*0.95

dfAges.iloc[0:12,2] = 0
dfAges.iloc[12:16,2] = 0.7

dfAges.iloc[16:20,2] = 0.90
dfAges.iloc[21:25,2] = 0.85
dfAges.iloc[25:30,2] = 0.85
dfAges.iloc[30:35,2] = 0.85
dfAges.iloc[35:40,2] = 0.85
dfAges.iloc[40:45,2] = 0.85
dfAges.iloc[45:50,2] = 0.90

# ['10D1. Personer fra årgang 1972-1976 og årgang 2002-2005', 87.0],
# ['10D2. Personer fra årgang 1977-1981 og årgang 1997-2001', 83.0],
# ['10D3. Personer fra årgang 1982-1986 og årgang 1992-1996', 76.0],
# ['10D4. Personer fra årgang 1987-1991', 76.0],
# ['Resterende', 20.0]

# Alternative:
dfAges.iloc[16:20,2] = 0.85
dfAges.iloc[21:25,2] = 0.80
dfAges.iloc[25:30,2] = 0.75
dfAges.iloc[30:35,2] = 0.70
dfAges.iloc[35:40,2] = 0.80
dfAges.iloc[40:45,2] = 0.80
dfAges.iloc[45:50,2] = 0.89

# # Alternative:
# dfAges.iloc[12:50,2] = 0.95

dfAges['VaccPred'] = dfAges['Antal']*dfAges['VaccProc']

totVaccPercent = dfAges.VaccPred.sum()/dfAges.Antal.sum()
totVaccPercent

0.7537630463463894

In [45]:
fig,ax1 = plt.subplots()

ax1.plot(dfAges.Alder,100*np.divide(dfAges.Antal.cumsum(),dfAges.Antal.sum()),'k')
ax1.set_ylabel('Andel af Danmarks befolkning [%]')
ax1.set_xlabel('Alder')
ax1.grid()

ax1.set_xlim([0,110])
ax1.set_ylim([0,100])

# ax2 = fig.add_axes([0.15,0.55,0.2,0.35])
ax2 = fig.add_axes([0.5,0.15,0.4,0.35])
ax2.plot(dfAges.Alder,100*np.divide(dfAges.Antal.cumsum(),dfAges.Antal.sum()),'k')


ax2.set_xlim([0,20])
ax2.set_ylim([0,25])
ax2.grid()

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-45-fe19ed2581aa>:20: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [46]:
# Try to plot difference from week to week

# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,ax1 = plt.subplots()


# for k in range(len(allGroups)-5,len(allGroups)):
for k in range(0,len(allGroups)):
    curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[k]]

    curLabel = allGroups[k]

    # curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[8]]
    # curLabel = allGroups[8]

    ageWeekDays = curdf.Ugenummer.apply(lambda x: np.datetime64(datetime.datetime.strptime(x + '-1', "%Y-W%W-%w")))

    ageVaccOnce = pd.to_numeric(curdf['Dækning førstegangsvaccinerede (kumulativt i %)']).values
    ageVaccFull = pd.to_numeric(curdf['Dækning færdigvaccinerede (kumulativt i %)']).values

    # ax1.plot(ageWeekDays,ageVaccOnce,'-*',label=curLabel)
    ax1.plot(ageWeekDays[1:],np.diff(ageVaccOnce),'-*',label=curLabel)

# ax1.set_ylim([0,100])
ax1.legend(fontsize=10)

ax1.set_xlim(left=np.datetime64('2021-03-01'))
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
# curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[0]]

# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

# for k in range(len(allGroups)-5,len(allGroups)-1):
#     curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[k]]

#     curLabel = allGroups[k]

#     ageWeekDays = curdf.Ugenummer.apply(lambda x: np.datetime64(datetime.datetime.strptime(x + '-1', "%Y-W%W-%w")))

#     ageVaccOnce = pd.to_numeric(curdf['Dækning førstegangsvaccinerede (kumulativt i %)'])
#     ageVaccFull = pd.to_numeric(curdf['Dækning færdigvaccinerede (kumulativt i %)'])
    
#     ax1.plot(ageWeekDays,ageVaccOnce,'-*',label=curLabel)
#     ax2.plot(ageWeekDays,ageVaccFull,'-*',label=curLabel)

# ax1.set_ylim([0,100])
# ax2.set_ylim([0,100])
# ax1.legend(fontsize=10)

# ax1.set_xlim(left=np.datetime64('2021-05-01'))
# plt.tight_layout()

df_weekly.tail(40)

,Ugenummer,Målgruppe,Dækning førstegangsvaccinerede (kumulativt i %),Dækning færdigvaccinerede (kumulativt i %)
499,2021-W29,5. Udvalgte patienter med særligt øget risiko,94.77,93.37
500,2021-W29,6. Udvalgte pårørende til personer med særligt...,90.36,89.93
501,2021-W29,7. Personer fra årgang 1937-1941,96.82,96.36
502,2021-W29,8. Personer fra årgang 1942-1946,96.84,96.35
503,2021-W29,9. Personer fra årgang 1947-1956,95.72,94.80
511,2021-W29,Resterende,5.03,0.00
512,2021-W30,1. Plejehjemsbeboere,96.16,94.38
521,2021-W30,10A. Personer fra årgang 1957-1961,93.03,91.27
522,2021-W30,10B. Personer fra årgang 1962-1966,91.37,88.69
523,2021-W30,10C. Personer fra årgang 1967-1971,89.48,84.94
